In [5]:
import pickle
import pandas as pd

In [6]:
from datetime import datetime

In [7]:
with open('../Data/allplayerFantasyGameLogs.pickle', 'rb') as handle:
  allplayerFantasyGameLogs = pickle.load(handle)

In [8]:
allplayerFantasyGameLogs.set_index('GAME_DATE', inplace=True)

In [9]:
allplayerFantasyGameLogs.columns

Index([   u'SEASON_ID',    u'Player_ID',      u'Game_ID',      u'MATCHUP',
                 u'WL',          u'MIN',          u'FGM',          u'FGA',
             u'FG_PCT',         u'FG3M',         u'FG3A',      u'FG3_PCT',
                u'FTM',          u'FTA',       u'FT_PCT',         u'OREB',
               u'DREB',          u'REB',          u'AST',          u'STL',
                u'BLK',          u'TOV',           u'PF',          u'PTS',
         u'PLUS_MINUS',        u'DouBL',        u'TriBL',       u'FanPTs',
           u'fullName',    u'position1',         u'Team', u'OpponentTeam',
           u'HomeGame'],
      dtype='object')

##Data Integration

We will split the allplayerGameLogs into training and test sets. And we need to use two helpful functions to extract player-level and team-level features to train the prediction model for fantasy points from a player. We also need another helpful function to aggregate corresponding information from the test set for the testing purpose.

In [11]:
def aggr(group):
        #group['NumGames'] = group['fullName'].count()
        group['AvgFanPTs'] = group['FanPTs'].mean()
        group['LastFanPTs'] = group['FanPTs'].tolist()[-1]
        group['AvgPTS'] = group['PTS'].mean()
        group['LastPT'] = group['PTS'].tolist()[-1]
        group['AvgMIN'] = group['MIN'].mean()
        group['LastMIN'] = group['MIN'].tolist()[-1]
        group['AvgFGM'] = group['FGM'].mean()
        group['LastFGM'] = group['FGM'].tolist()[-1]
        group['AvgFGA'] = group['FGA'].mean()
        group['LastFGA'] = group['FGA'].tolist()[-1]
        group['AvgFG3M'] = group['FG3M'].mean()
        group['LastFG3M'] = group['FG3M'].tolist()[-1]
        group['AvgFG3A'] = group['FG3A'].mean()
        group['LastFG3A'] = group['FG3A'].tolist()[-1]
        group['AvgREB'] = group['REB'].mean()
        group['LastREB'] = group['REB'].tolist()[-1]
        group['AvgAST'] = group['AST'].mean()
        group['LastAST'] = group['AST'].tolist()[-1]
        group['AvgSTL'] = group['STL'].mean()
        group['AvgTOV'] = group['TOV'].mean() 
        group['LastTOV'] = group['TOV'].tolist()[-1]
        group['AvgPF'] = group['PF'].mean()
        group['LastPT'] = group['PF'].tolist()[-1]
        group['AvgPLUS_MINUS'] = group['PLUS_MINUS'].mean()
        group['LastPLUS_MINUS'] = group['PLUS_MINUS'].tolist()[-1]
        #group['NumDouBL'] = group['DouBL'].sum()
        #group['NumTriBL'] = group['TriBL'].sum()

        group['Last3GameAvgFanPTs'] = group['FanPTs'][:3].mean()
        group['Last3GameAvgMIN'] = group['MIN'][:3].mean()
        group['Last3GameAvgPTS'] = group['PTS'][:3].mean()
    
def aggr_stats(date,allplayerFantasyGameLogs):
    interest_columns = ['fullName','Player_ID','Team','position1','MIN','PTS','FGM','FGA', 'FG3M','FG3A', \
                        'REB','AST','STL','TOV','PF','PLUS_MINUS','DouBL','TriBL','FanPTs']
    tmp = allplayerFantasyGameLogs.ix['2015-10-27':date]
    
    playerID_tmp = tmp.reset_index().copy()
    tmp.grouped = playerID_tmp[interest_columns].groupby('Player_ID')
    
    Newdf = tmp.grouped.apply(aggr)
    Newdf = Newdf.drop(['MIN','PTS','FGM','FGA', 'FG3M','FG3A','REB','AST','STL','TOV','PF','PLUS_MINUS','DouBL','TriBL','FanPTs'],axis=1)
    Newdf.drop_duplicates(inplace=True)
    
    bins = [-10, 10, 20, 30, 40, 100]
    group_names = ['benchPlayer','belowAvg','average','advanced','top']
    Newdf['Rank']= pd.cut(Newdf['AvgFanPTs'],bins,labels=group_names)
    
    return(Newdf)

In [12]:
def aggr_teamVSteam(group):
        group['TeamStdVSFanPTs'] = group['FanPTs'].std()
        group['TeamAvgVSFanPTs'] = group['FanPTs'].mean()
        group['TeamMaxVSFanPTs'] = group['FanPTs'].max()
        return group

def aggr_team(group):
        group['TeamStdFanPTs'] = group['TeamStdVSFanPTs'].mean()
        group['TeamAvgFanPTs'] = group['TeamAvgVSFanPTs'].mean()
        group['TeamMaxFanPTs'] = group['TeamMaxVSFanPTs'].mean()
        return group    

def generate_team_features(playerGameLogs, playerFeatureTable, date):
    tmp = playerGameLogs['2015-10-27': date]
    tmp = tmp.reset_index()
    bad_players = playerFeatureTable[playerFeatureTable.Rank=='benchPlayer']['Player_ID']
    interest_cols = ['fullName','Player_ID','Team','OpponentTeam','position1','FanPTs','MIN']
    tmp = tmp[interest_cols]
    tmp = tmp[~tmp['Player_ID'].isin(bad_players)]
    
    newdf = tmp.copy()
    newdf_grouped = newdf.groupby(['Team','OpponentTeam'])
        
    Newdf = newdf_grouped.apply(aggr_teamVSteam)
    Newdf.drop(['fullName','Player_ID','MIN','FanPTs','position1'],inplace=True,axis=1)
    Newdf.drop_duplicates(['Team','OpponentTeam'],inplace=True)
    
    Newdf.drop('OpponentTeam',axis=1,inplace=True)
    
    Newdf2 = Newdf.copy()
    Newdf2_grouped = Newdf2.groupby('Team')
    
    Newdf_overall = Newdf2_grouped.apply(aggr_team)
    Newdf_overall.drop(['TeamStdVSFanPTs','TeamAvgVSFanPTs','TeamMaxVSFanPTs'],inplace=True,axis=1)
    Newdf_overall.drop_duplicates('Team',inplace=True)
    
    return(Newdf_overall)

In [13]:
def drop_y(df):
    # list comprehension of the cols that end with '_y'
    to_drop = [x for x in df if x.endswith('_y')]
    df.drop(to_drop, axis=1, inplace=True)

def rename_x(df):
    for col in df:
        if col.endswith('_x'):
            df.rename(columns={col:col.rstrip('_x')}, inplace=True)

In [14]:
def get_train_test(train_date, test_date): #format like'2/10/2016'
    train_date_index = pd.date_range(start='11/10/2015', end=train_date, freq='D')
    train_df = pd.DataFrame()
    
    alldates = allplayerFantasyGameLogs.index
    trydates = pd.date_range(start='10/27/2015', end='2/28/2016', freq='D')
    s = set(alldates)
    nodates = [x for x in trydates if x not in s]
    
    for idx in train_date_index:
        tmp_idx = idx+1
        if tmp_idx not in nodates and idx not in nodates:
            #aggregate the statistics from players -> player-level features
            trainLogs = allplayerFantasyGameLogs.ix['2015-10-27':idx]
            train_player_df = aggr_stats(idx,trainLogs)   
            #next we need to collect the player's next game Fantasy Points.
            next_date = idx + 1
            tmpLogs = allplayerFantasyGameLogs[['fullName', 'Player_ID','Team','OpponentTeam','HomeGame','FanPTs']].ix[next_date]
            tmpLogs.rename(columns={'FanPTs':'NewGameFanPTs'},inplace=True)
            #join the tmpLogs and player festure table by Player_ID, which is based on the players on a new game day
            newgame_df = pd.merge(tmpLogs,train_player_df,how='inner',on='Player_ID')
            drop_y(newgame_df)
            rename_x(newgame_df)

            #get the team features table 
            train_team_df = generate_team_features(allplayerFantasyGameLogs, train_player_df, idx)
            newgame_df = pd.merge(newgame_df,train_team_df,how='left',on='Team')
            train_df = pd.concat([train_df,newgame_df],axis=0)

    test_date_index = pd.date_range(start=train_date, end=test_date, freq='D')[1:]
    start_test_date = pd.date_range(start=train_date, end=test_date, freq='D')[0]
    test_df = pd.DataFrame()
    for idx in test_date_index:
        tmp_idx = idx+1
        if tmp_idx not in nodates and idx not in nodates:
            #aggregate the statistics from players -> player-level features
            testLogs = allplayerFantasyGameLogs.ix[start_test_date:idx]
            test_player_df = aggr_stats(idx,testLogs)   
            #next we need to collect the player's next game Fantasy Points.
            next_date = idx + 1
            tmpLogs = allplayerFantasyGameLogs[['fullName', 'Player_ID','Team','OpponentTeam','HomeGame','FanPTs']].ix[next_date]
            tmpLogs.rename(columns={'FanPTs':'NewGameFanPTs'},inplace=True)
            #join the tmpLogs and player festure table by Player_ID, which is based on the players on a new game day
            newgame_df = pd.merge(tmpLogs,train_player_df,how='inner',on='Player_ID')
            drop_y(newgame_df)
            rename_x(newgame_df)

            #get the team features table 
            test_team_df = generate_team_features(allplayerFantasyGameLogs, test_player_df, idx)
            newgame_df = pd.merge(newgame_df,test_team_df,how='left',on='Team')
            test_df = pd.concat([test_df,newgame_df],axis=0)                
        
    return(train_df, test_df)

In [15]:
train_set, test_set = get_train_test('2/18/2016', '2/28/2016')

In [14]:
train_set.shape

NameError: name 'train_set' is not defined

In [17]:
test_set.shape

(1530, 38)

In [18]:
train_set.columns

Index([          u'fullName',          u'Player_ID',               u'Team',
             u'OpponentTeam',           u'HomeGame',      u'NewGameFanPTs',
                u'position1',          u'AvgFanPTs',         u'LastFanPTs',
                   u'AvgPTS',             u'LastPT',             u'AvgMIN',
                  u'LastMIN',             u'AvgFGM',            u'LastFGM',
                   u'AvgFGA',            u'LastFGA',            u'AvgFG3M',
                 u'LastFG3M',            u'AvgFG3A',           u'LastFG3A',
                   u'AvgREB',            u'LastREB',             u'AvgAST',
                  u'LastAST',             u'AvgSTL',             u'AvgTOV',
                  u'LastTOV',              u'AvgPF',      u'AvgPLUS_MINUS',
           u'LastPLUS_MINUS', u'Last3GameAvgFanPTs',    u'Last3GameAvgMIN',
          u'Last3GameAvgPTS',               u'Rank',      u'TeamStdFanPTs',
            u'TeamAvgFanPTs',      u'TeamMaxFanPTs'],
      dtype='object')

In [19]:
with open('../Data/train_set_02_29.pickle', 'wb') as handle:
  pickle.dump(train_set, handle)

In [20]:
with open('../Data/test_set_02_29.pickle', 'wb') as handle:
  pickle.dump(test_set, handle)

##Prepocessing

In [21]:
with open('../Data/train_set_02_29.pickle', 'rb') as handle:
  train_set = pickle.load(handle)

In [22]:
with open('../Data/test_set_02_29.pickle', 'rb') as handle:
  test_set = pickle.load(handle)

In [23]:
test_set.shape

(1530, 38)

In [24]:
train_set.head()

,fullName,Player_ID,Team,OpponentTeam,HomeGame,NewGameFanPTs,position1,AvgFanPTs,LastFanPTs,AvgPTS,...,AvgPF,AvgPLUS_MINUS,LastPLUS_MINUS,Last3GameAvgFanPTs,Last3GameAvgMIN,Last3GameAvgPTS,Rank,TeamStdFanPTs,TeamAvgFanPTs,TeamMaxFanPTs
0,Jamal Crawford,2037,LAC,DAL,0,12.50,SG,15.607143,15.25,10.000000,...,0.714286,1.714286,19,15.750000,20.333333,12.000000,belowAvg,13.805475,23.300926,49.50000
1,DeAndre Jordan,201599,LAC,DAL,0,23.75,C,36.821429,38.50,10.142857,...,3.000000,7.857143,-12,41.083333,29.666667,9.666667,advanced,13.805475,23.300926,49.50000
2,JJ Hickson,201581,DEN,MIL,1,19.50,C,21.350000,44.25,10.000000,...,2.400000,2.200000,4,18.416667,17.666667,8.666667,average,10.420933,21.045960,38.25000
3,Frank Kaminsky,1626163,CHA,NYK,1,12.00,C,6.250000,-0.50,3.142857,...,0.428571,0.571429,-5,5.166667,8.666667,2.666667,benchPlayer,8.812792,23.941592,38.12500
4,Mike Conley,201144,MEM,GSW,1,24.00,PG,28.593750,32.50,13.750000,...,1.500000,-6.750000,8,32.833333,30.333333,14.333333,average,9.556530,19.327778,35.59375


In [25]:
#Combine into data:
train_set['source']= 'train'
test_set['source'] = 'test'
data=pd.concat([train_set, test_set],ignore_index=True)
data.shape

(16168, 39)

###Check Missing values

In [26]:
data.apply(lambda x: sum(x.isnull()))

fullName              0
Player_ID             0
Team                  0
OpponentTeam          0
HomeGame              0
NewGameFanPTs         0
position1             0
AvgFanPTs             0
LastFanPTs            0
AvgPTS                0
LastPT                0
AvgMIN                0
LastMIN               0
AvgFGM                0
LastFGM               0
AvgFGA                0
LastFGA               0
AvgFG3M               0
LastFG3M              0
AvgFG3A               0
LastFG3A              0
AvgREB                0
LastREB               0
AvgAST                0
LastAST               0
AvgSTL                0
AvgTOV                0
LastTOV               0
AvgPF                 0
AvgPLUS_MINUS         0
LastPLUS_MINUS        0
Last3GameAvgFanPTs    0
Last3GameAvgMIN       0
Last3GameAvgPTS       0
Rank                  0
TeamStdFanPTs         0
TeamAvgFanPTs         0
TeamMaxFanPTs         0
source                0
dtype: int64

###Look at categories of all object variables

In [27]:
var = ['Team','OpponentTeam', 'position1','Rank']
for v in var:
    print '\nFrequency count for variable %s'%v
    print data[v].value_counts()


Frequency count for variable Team
SAS    596
DAL    586
BOS    578
PHI    576
NYK    570
MEM    570
BKN    568
POR    566
MIN    559
ATL    556
UTA    555
OKC    543
GSW    543
HOU    542
DEN    538
LAL    538
ORL    535
LAC    532
IND    531
PHX    526
DET    517
CHA    516
MIL    516
WAS    513
MIA    512
TOR    508
CLE    506
NOP    506
CHI    484
SAC    482
Name: Team, dtype: int64

Frequency count for variable OpponentTeam
PHI    577
LAL    572
BKN    570
DAL    567
BOS    565
DET    560
PHX    560
DEN    559
POR    559
MIN    548
NYK    547
UTA    544
SAS    544
MIL    541
HOU    540
ATL    539
MEM    537
IND    532
CHA    530
ORL    529
WAS    527
NOP    527
OKC    522
CLE    520
LAC    518
SAC    516
TOR    512
MIA    511
GSW    511
CHI    484
Name: OpponentTeam, dtype: int64

Frequency count for variable position1
SG    3517
PG    3438
PF    3353
SF    3037
C     2823
Name: position1, dtype: int64

Frequency count for variable Rank
belowAvg       6048
average        4209
benc

###Numerical Coding and One-Hot Coding

In [28]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
var_to_encode = ['Team','OpponentTeam','position1','HomeGame','Rank']
for col in var_to_encode:
    data[col] = le.fit_transform(data[col])

In [29]:
data = pd.get_dummies(data, columns=var_to_encode)
data.columns

Index([     u'fullName',     u'Player_ID', u'NewGameFanPTs',     u'AvgFanPTs',
          u'LastFanPTs',        u'AvgPTS',        u'LastPT',        u'AvgMIN',
             u'LastMIN',        u'AvgFGM',
       ...
         u'position1_2',   u'position1_3',   u'position1_4',    u'HomeGame_0',
          u'HomeGame_1',        u'Rank_0',        u'Rank_1',        u'Rank_2',
              u'Rank_3',        u'Rank_4'],
      dtype='object', length=106)

###Separate train & test:

In [30]:
train = data.loc[data['source']=='train']
test = data.loc[data['source']=='test']

In [31]:
train.drop('source',axis=1,inplace=True)
test.drop('source',axis=1,inplace=True)

/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [32]:
train.to_csv('../Data/train_modified_0229.csv',index=False)
test.to_csv('../Data/test_modified_0229.csv',index=False)